# <center> Innoscape - Telecom Paristech - Projet fil rouge </center>
# <center> Mapping Groupes de produits </center>

In [1]:
# Librairies 
import pandas as pd
import numpy as np
from scipy import stats as scs
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import FrenchStemmer
import re
import random
from Preprocessing2 import *
import gensim.models as models
from gensim import corpora

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 60)

#chemin = "../Data/"
chemin = "C:/Users/JN/Desktop/Arbeit/fil_rouge/github/Data/"
chemin_val = "/Users/valentinphetchanpheng/Documents/MS BGD Telecom ParisTech/Projet Fil Rouge/"

C:\Users\JN\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Importation des données et reformatage

In [2]:
# Importation des données vendeurs
df_seller_origin = pd.read_csv(chemin + "t3_all_fr_prices_posweekly_prod_without_price.csv", sep = ",")

#Création d'un df allégé pour le travail - on ne retient que les colonnes identifiées comme utiles pour l'analyse à ce stade
columns_to_keep = ['t3_all_fr_prices_posweekly_prod.productfamily_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory4_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory3_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory2_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory1_seller',
                   't3_all_fr_prices_posweekly_prod.productseller_name',
                   't3_all_fr_prices_posweekly_prod.id_source',
                   't3_all_fr_prices_posweekly_prod.brand_seller']

df_seller = df_seller_origin[columns_to_keep]

#on raccourcit un peu les noms des colonnes en ôtant les préfixes redondants
shortened_names = map(lambda string: string.replace('t3_all_fr_prices_posweekly_prod.',''), columns_to_keep)
col_name_dict = dict(zip(columns_to_keep, shortened_names))
df_seller.rename(columns=col_name_dict, inplace=True)

In [3]:
# Importation des données références catégories Innoscape
df_ref_origin = pd.read_csv(chemin + "2018.11.21_innoscape_internal_architecture.csv")
df_ref = df_ref_origin.drop("Date", axis=1)

#On garde que la colonne correspondante à la famille de produit de référence finale
columns_to_keep = ['innoscape_ref_productfamily_seller']
df_ref = df_ref[columns_to_keep]
df_ref = df_ref.drop_duplicates()

In [4]:
df_seller = df_seller.dropna(how='all', subset=['productfamily_seller',
                                                'prodcategory4_seller',
                                                'prodcategory3_seller',
                                                'prodcategory2_seller',
                                                'prodcategory1_seller',
                                                'productseller_name'])
df_seller = df_seller.fillna("")

# Retrait des caractères spéciaux et de la ponctuation
dictionary = {'à':'a','â':'a','ä':'a',
              'é':'e','è':'e','ê':'e','ë':'e',
              'ï':'i','î':'i',
              'ô':'o','ö':'o',
              'ù':'u','û':'u',
              'ç':'c',
              ',':' ', "'":" ", ";":" ", ":":" ", "\?":" ", "!":" ", "\.":'', "  ":" ", 
              "\(":"","\)":"","\[":"","\]":"","-":" ","\+":"","/":"","\*":"","–":" ",
              "&":"et"
              }


In [5]:
# Importation de la liste de stop words
f = open("stop_words.txt", "r")
stop_words = set(f.read().split('\n'))

g = open("marques.txt", "r")
marques = set(g.read().split('\n'))

stop_words = stop_words | marques

In [6]:
#preprocessing des données
df_ref=preprocessing(df_ref,df_ref.columns,dictionary, stop_words)
list_new_col = ['productfamily_seller', 'prodcategory4_seller', 'prodcategory3_seller', 'prodcategory2_seller', 'prodcategory1_seller', 'productseller_name']
df_seller = preprocessing(df_seller, list_new_col, dictionary, stop_words)

In [7]:
df_seller.head(10)

productfamily_seller   prodcategory4_seller  \
0     Colles à carrelage     Colles à carrelage   
1                                                 
2                                                 
3                 Mastic                 Mastic   
4                  Cires                  Cires   
5                                                 
6      Perceuse-visseuse              Perceuses   
7  Produits d'étanchéité  Produits d'étanchéité   
8              Arroseurs              Arroseurs   
9                                                 

               prodcategory3_seller                      prodcategory2_seller  \
0                Colles et adhésifs                             Quincaillerie   
1                                                                               
2  Accessoires pour lave-vaisselles                               Accessoires   
3                Colles et adhésifs                             Quincaillerie   
4  Peintures, teintures et solvants  Peintures, outils et traitement des murs   
5                                    Peintures, outils et traitement des murs   
6         Outillage électroportatif       Outillage à main et électroportatif   
7                Colles et adhésifs                             Quincaillerie   
8               Matériel d'arrosage                                 Jardinage   
9                                                                               

  prodcategory1_seller                                 productseller_name  \
0            Bricolage  SikaCeram Xtra - Colle à carrelage intérieur e...   
1                       Ariston 3200362 scaldabagno elettrico PRO 100 H/3   
2  Gros électroménager  AEG Panier À Couverts Lave-vaisselle Cage & Ha...   
3            Bricolage              Colle-mastic pour joints d'étanchéité   
4            Bricolage  Sikagard Protection Sol MAT - Imperméabilisant...   
5            Bricolage  Sikagard Protection Toiture inclinée - Protect...   
6            Bricolage  Perceuse-visseuse RYOBI 18V OnePlus - 1 batter...   
7            Bricolage  Sika Multiseal - Bande d'étanchéité autocollan...   
8               Jardin            Black and Decker 34345 Arroseur Impulse   
9                       Ryobi-RXB01 Cortasetos-Écartement des dents La...   

  id_source      brand_seller productfamily_seller_formatted  \
0    AMZ_FR     SIKAFRANCESAS             colles a carrelage   
1    AMZ_FR  ARISTONTHERMOSPA                                  
2    AMZ_FR               AEG                                  
3    AMZ_FR     SIKAFRANCESAS                         mastic   
4    AMZ_FR     SIKAFRANCESAS                          cires   
5    AMZ_FR     SIKAFRANCESAS                                  
6    AMZ_FR             RYOBI              perceuse visseuse   
7    AMZ_FR     SIKAFRANCESAS          produits d etancheite   
8    AMZ_FR       BLACKDECKER                      arroseurs   
9    AMZ_FR             RYOBI                                  

  prodcategory4_seller_formatted    prodcategory3_seller_formatted  \
0             colles a carrelage                colles et adhesifs   
1                                                                    
2                                 accessoires pour lave vaisselles   
3                         mastic                colles et adhesifs   
4                          cires   peintures teintures et solvants   
5                                                                    
6                      perceuses         outillage electroportatif   
7          produits d etancheite                colles et adhesifs   
8                      arroseurs               materiel d arrosage   
9                                                                    

            prodcategory2_seller_formatted prodcategory1_seller_formatted  \
0                            quincaillerie                      bricolage   
1                                                                       

In [8]:
cols_to_keep = ['productfamily_seller',
                'prodcategory4_seller',
                'prodcategory3_seller',
                'prodcategory2_seller',
                'prodcategory1_seller',
                'productseller_name',
                'id_source',
                'productfamily_seller_stem',
                'prodcategory4_seller_stem',
                'prodcategory3_seller_stem',
                'prodcategory2_seller_stem',
                'prodcategory1_seller_stem',
                'productseller_name_stem']

df2 = df_seller[cols_to_keep]
df2.rename(columns={'productfamily_seller_stem':"prod_fam",
                'prodcategory4_seller_stem':'cat4',
                'prodcategory3_seller_stem':'cat3',
                'prodcategory2_seller_stem':'cat2',
                'prodcategory1_seller_stem':'cat1',
                'productseller_name_stem':'description'}, inplace=True)

In [9]:
df2.head()

productfamily_seller prodcategory4_seller              prodcategory3_seller  \
0   Colles à carrelage   Colles à carrelage                Colles et adhésifs   
1                                                                               
2                                            Accessoires pour lave-vaisselles   
3               Mastic               Mastic                Colles et adhésifs   
4                Cires                Cires  Peintures, teintures et solvants   

                       prodcategory2_seller prodcategory1_seller  \
0                             Quincaillerie            Bricolage   
1                                                                  
2                               Accessoires  Gros électroménager   
3                             Quincaillerie            Bricolage   
4  Peintures, outils et traitement des murs            Bricolage   

                                  productseller_name id_source  \
0  SikaCeram Xtra - Colle à carrelage intérieur e...    AMZ_FR   
1  Ariston 3200362 scaldabagno elettrico PRO 100 H/3    AMZ_FR   
2  AEG Panier À Couverts Lave-vaisselle Cage & Ha...    AMZ_FR   
3              Colle-mastic pour joints d'étanchéité    AMZ_FR   
4  Sikagard Protection Sol MAT - Imperméabilisant...    AMZ_FR   

           prod_fam              cat4                       cat3  \
0  [carrelag, coll]  [carrelag, coll]              [coll, adhes]   
1                []                []                         []   
2                []                []  [vaissel, accessoir, lav]   
3          [mastic]          [mastic]              [coll, adhes]   
4             [cir]             [cir]   [peintur, solv, teintur]   

                           cat2                  cat1  \
0                 [quincailler]            [bricolag]   
1                            []                    []   
2                   [accessoir]  [electromenag, gros]   
3                 [quincailler]            [bricolag]   
4  [trait, peintur, mur, outil]            [bricolag]   

                                         description  
0  [blanc, carrelag, max, xtra, interieur, coll, ...  
1                      [scaldabagno, pro, elettrico]  
2  [compart, couvert, handl, cag, pani, vaissel, ...  
3                   [coll, mastic, etancheit, joint]  
4  [effet, sol, impermeabilis, protect, mat, sika...

In [10]:
#concaténation de tous les champs pertinents en un seul texte par produit
# df2['prod_info'] = df2.prod_fam + df2.cat4 +df2.description
# df2['prod_info'] = df2['prod_info'].apply(lambda x: ' '.join(x))
# df2['prod_info']

df2['prod_info'] = df2.description

## TF-IDF

In [48]:
#tfidf version gensim
texts = [document for document in df2['prod_info']]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus)
vectors = tfidf[corpus]

In [49]:
vectors[100]

[(331, 0.39068397660094234),
 (398, 0.44221684947607476),
 (399, 0.6737648033524819),
 (400, 0.44480476417201303)]

## Embeddings

### 1-Latent semantic indexing

#### 1.1 - Détermination de la dimension de l'embedding

In [24]:
# n_dim = 1000
# lsi = models.LsiModel(vectors, id2word=dictionary, num_topics=n_dim)
# corpus_lsi = lsi[vectors]

In [50]:
#affichage des valeurs propres pour déterminer le bon nombre de topics
# eigenvalues = lsi.projection.s
# plt.scatter(range(n_dim),eigenvalues)
# plt.xlabel('topic rank')
# plt.ylabel('topic eigenvalue')
# plt.show()

In [75]:
# embedding lda
lda = models.LdaModel(corpus = vectors,
                      id2word=dictionary,
                      num_topics=10,
                      chunksize = 3000,
                      alpha='auto',
                      random_state=0
                      )
corpus_lda = lda[vectors]

In [76]:
lda.show_topics(100)

[(0,
  '0.038*"seau" + 0.035*"silvacan" + 0.030*"littoral" + 0.024*"sous" + 0.022*"omeg" + 0.021*"cuivr" + 0.019*"poincon" + 0.018*"band" + 0.017*"xmm" + 0.016*"atlant"'),
 (1,
  '0.058*"canal" + 0.024*"languedoc" + 0.018*"verr" + 0.018*"pot" + 0.018*"tuil" + 0.014*"litr" + 0.013*"mid" + 0.012*"tig" + 0.012*"fibr" + 0.011*"primair"'),
 (2,
  '0.028*"mass" + 0.020*"toitur" + 0.017*"fin" + 0.016*"aerosol" + 0.016*"terr" + 0.016*"cuit" + 0.015*"peintur" + 0.015*"angle" + 0.015*"grain" + 0.015*"impermeabilis"'),
 (3,
  '0.031*"rouleau" + 0.028*"sac" + 0.022*"vieux" + 0.019*"enduit" + 0.017*"pay" + 0.017*"plat" + 0.013*"graphit" + 0.013*"clous" + 0.013*"color" + 0.012*"liss"'),
 (4,
  '0.044*"section" + 0.043*"lantern" + 0.025*"teint" + 0.024*"closoir" + 0.019*"gallean" + 0.013*"sud" + 0.011*"ventil" + 0.011*"sac" + 0.011*"plein" + 0.010*"douill"'),
 (5,
  '0.030*"ros" + 0.022*"perfor" + 0.021*"sd" + 0.020*"ventile" + 0.018*"foret" + 0.017*"occitan" + 0.016*"mastic" + 0.016*"boit" + 0.014*"